In [ ]:
import os
import ipywidgets as widgets
import cv2
import numpy as np
import matplotlib.pyplot as plt

rng = np.random.default_rng()
%matplotlib inline

# Import Video Files

In [ ]:
infile = widgets.Text(
    value='',
    placeholder='filepath',
    description='Input file:',
    disabled=False   
)
infile

In [ ]:
def sample_from_video(filepath,
                      target_sample_size,
                      tail_cut_frames = 30, # fudge factor to avoid falling off the end of the video
                      out_dir = "./TEMP",
                      out_prefix = "frame",
                     ):
    """
    Opens the given video file, takes a sample of frames, writes the sample to out_dir.
    Returns the list of files written or None if error.
    """
    v = cv2.VideoCapture(filepath)
    if not v.isOpened():
        print(f"Failed to read video from {filepath}")
        return None
        
    n_frames = int(v.get(cv2.CAP_PROP_FRAME_COUNT))
    print(f"Opened video with approx. {n_frames} frames")
    
    sampled_idxs = rng.choice(np.arange(n_frames - tail_cut_frames), size = target_sample_size, replace = False)
    sampled_idxs.sort()
    
    frames = []
    i = 0
    done = False
    for idx in sampled_idxs:
        while True:
            ret, frame = v.read()
            i += 1
            if not ret:
                print(f"Failed to read video at frame {i}: sample might be smaller than requested due to error in opencv's frame count estimate")
                done = True
                break
            if i > idx:
                break
    
        if done:
            break
            
        frames.append(frame)
    print(f"Writing {len(frames)} frames to {out_dir}.")
    try:
        os.mkdir(out_dir)
    except (FileExistsError, FileNotFoundError) as e:
        print(f"Failed to make a new directory at {out_dir}: {e}")
        return None

    filenames = []
    for i, f in enumerate(frames):
        filename = f"{out_dir}/{out_prefix}_{i}.jpg"
        cv2.imwrite(filename, f)
        filenames.append(filename)

    return filenames

filepath = infile.value
if not os.path.isfile(filepath):
    print(f"Failed to read from \"{filepath}\"")
else:
    filenames = sample_from_video(filepath, target_sample_size = 5)

In [ ]:
def view_image(i):
    file = open(filenames[i], "rb")
    image = file.read()
    img = widgets.Image(
        value=image,
        format='jpeg',
    )
    return img
widgets.interact(view_image, i=(0,len(filenames)-1))

# Label

# Train